In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

In [7]:
# Model definition
class CIFAR10_MLP(nn.Module):
    def __init__(self, input_dim=3*32*32, hidden_dims=[2048, 1024, 512, 256], num_classes=10, dropout=0.5):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for hdim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hdim))
            layers.append(nn.BatchNorm1d(hdim))
            layers.append(nn.ReLU(inplace=True))
            layers.append(nn.Dropout(dropout))
            prev_dim = hdim
        layers.append(nn.Linear(prev_dim, num_classes))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # flatten
        return self.net(x)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data transforms: normalize CIFAR-10 images
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), 
                         (0.247, 0.243, 0.261))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), 
                         (0.247, 0.243, 0.261))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
model = CIFAR10_MLP().to(device)
model = torch.jit.script(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [5]:
from utils import train

train_metrics, val_metrics, test_metrics = train(model, train_loader, None, test_loader, 30, optimizer, criterion)

Epoch: 1 Total_Time: 2.2621 Average_Time_per_batch: 0.0058 Train_Accuracy: 0.2778 Train_Loss: 1.9698 
Epoch: 2 Total_Time: 1.7568 Average_Time_per_batch: 0.0045 Train_Accuracy: 0.3492 Train_Loss: 1.7906 
Epoch: 3 Total_Time: 1.8371 Average_Time_per_batch: 0.0047 Train_Accuracy: 0.3748 Train_Loss: 1.7267 
Epoch: 4 Total_Time: 1.7929 Average_Time_per_batch: 0.0046 Train_Accuracy: 0.3913 Train_Loss: 1.6829 
Epoch: 5 Total_Time: 2.0411 Average_Time_per_batch: 0.0052 Train_Accuracy: 0.4054 Train_Loss: 1.6491 
Epoch: 6 Total_Time: 1.9651 Average_Time_per_batch: 0.0050 Train_Accuracy: 0.4134 Train_Loss: 1.6267 
Epoch: 7 Total_Time: 1.8747 Average_Time_per_batch: 0.0048 Train_Accuracy: 0.4218 Train_Loss: 1.6057 
Epoch: 8 Total_Time: 1.9524 Average_Time_per_batch: 0.0050 Train_Accuracy: 0.4284 Train_Loss: 1.5874 
Epoch: 9 Total_Time: 1.3347 Average_Time_per_batch: 0.0034 Train_Accuracy: 0.4348 Train_Loss: 1.5715 
Epoch: 10 Total_Time: 1.7597 Average_Time_per_batch: 0.0045 Train_Accuracy: 0.4363

In [6]:
from spn_3.spn import SPN as SPN_3
    
model_3 = SPN_3(3*32*32, 500, 10, False).cuda()
#model_3.compile()

In [7]:
model_3 = torch.jit.trace(model_3, torch.randn(128, 3*32*32).cuda())
optimizer = optim.Adam(model_3.parameters())
criterion = nn.CrossEntropyLoss()

In [8]:
train_metrics_3, val_metrics_3, test_metrics_3 = train(model_3, train_loader, None, test_loader, 30, optimizer, criterion)

Epoch: 1 Total_Time: 112.6011 Average_Time_per_batch: 0.2880 Train_Accuracy: 0.3247 Train_Loss: 1.9529 
Epoch: 2 Total_Time: 47.9317 Average_Time_per_batch: 0.1226 Train_Accuracy: 0.3782 Train_Loss: 1.7925 
Epoch: 3 Total_Time: 47.2408 Average_Time_per_batch: 0.1208 Train_Accuracy: 0.3942 Train_Loss: 1.7465 
Epoch: 4 Total_Time: 47.2769 Average_Time_per_batch: 0.1209 Train_Accuracy: 0.4075 Train_Loss: 1.7088 
Epoch: 5 Total_Time: 47.5081 Average_Time_per_batch: 0.1215 Train_Accuracy: 0.4189 Train_Loss: 1.6820 
Epoch: 6 Total_Time: 47.6612 Average_Time_per_batch: 0.1219 Train_Accuracy: 0.4234 Train_Loss: 1.6580 
Epoch: 7 Total_Time: 47.9120 Average_Time_per_batch: 0.1225 Train_Accuracy: 0.4267 Train_Loss: 1.6431 
Epoch: 8 Total_Time: 47.9463 Average_Time_per_batch: 0.1226 Train_Accuracy: 0.4302 Train_Loss: 1.6342 
Epoch: 9 Total_Time: 47.3590 Average_Time_per_batch: 0.1211 Train_Accuracy: 0.4364 Train_Loss: 1.6162 
Epoch: 10 Total_Time: 47.3619 Average_Time_per_batch: 0.1211 Train_Accur

In [28]:
import torch
from torch import nn

hidden_dims = [2048, 1024, 512, 256]
total = sum(hidden_dims)

blocks = len(hidden_dims)
features = 3 * 32 * 32
neural_blocks = []
for dim in hidden_dims:
    std_dev = torch.sqrt(torch.tensor(1 / features)).to(device)
    neural_blocks.append(torch.randn(dim, features).to(device) * std_dev)
    features += dim

feature_blocks = []
features_start = 0
for i in range(len(neural_blocks)):
    features_end = neural_blocks[i].shape[1]
    block = neural_blocks[i][:, features_start:]
    for j in range(i + 1, len(neural_blocks)):
        block = torch.cat((block, neural_blocks[j][:, features_start:features_end]), dim=0)
    feature_blocks.append(nn.Parameter(block))
    features_start = features_end

biases = biases = nn.Parameter(torch.empty(total).uniform_(0.0, 1.0)).to(device)

In [29]:
from spn_2.spn import SPN as SPN_3
    
model_3 = SPN_3(3*32*32, 500, 10, False).cuda()
model_3.weights.extend(feature_blocks)
model_3.biases = biases

In [31]:
#model_3 = torch.jit.trace(model_3, torch.randn(128, 3*32*32).cuda())
optimizer = optim.Adam(model_3.parameters())
criterion = nn.CrossEntropyLoss()

In [33]:
from utils import train
train_metrics_3, val_metrics_3, test_metrics_3 = train(model_3, train_loader, None, test_loader, 50, optimizer, criterion)

Epoch: 1 Total_Time: 2.1577 Average_Time_per_batch: 0.0055 Train_Accuracy: 0.3155 Train_Loss: 2.2739 
Epoch: 2 Total_Time: 1.7335 Average_Time_per_batch: 0.0044 Train_Accuracy: 0.3697 Train_Loss: 1.9101 
Epoch: 3 Total_Time: 1.6517 Average_Time_per_batch: 0.0042 Train_Accuracy: 0.3816 Train_Loss: 1.9002 
Epoch: 4 Total_Time: 1.7158 Average_Time_per_batch: 0.0044 Train_Accuracy: 0.3919 Train_Loss: 1.9126 
Epoch: 5 Total_Time: 1.5809 Average_Time_per_batch: 0.0040 Train_Accuracy: 0.3999 Train_Loss: 1.8769 
Epoch: 6 Total_Time: 1.6350 Average_Time_per_batch: 0.0042 Train_Accuracy: 0.4039 Train_Loss: 1.8624 
Epoch: 7 Total_Time: 1.7054 Average_Time_per_batch: 0.0044 Train_Accuracy: 0.4159 Train_Loss: 1.8457 
Epoch: 8 Total_Time: 1.6367 Average_Time_per_batch: 0.0042 Train_Accuracy: 0.4205 Train_Loss: 1.8634 
Epoch: 9 Total_Time: 1.7133 Average_Time_per_batch: 0.0044 Train_Accuracy: 0.4207 Train_Loss: 1.8766 
Epoch: 10 Total_Time: 1.7132 Average_Time_per_batch: 0.0044 Train_Accuracy: 0.4289